In [9]:
#importing stuff and setting global variables
import cv2 as cv# for realtime processing
import math
import mediapipe as mp# fro AI 
#import pyautogui as py#yeh screenshot k liay hai
# from mediapipe_model_maker import gesture_recognizer

The Code below i for creating the task or i would say a medol instance for recognizing guestures for a curstom data set
First piece of code is to load data from a folder into variable data and than split it into testing and training data
Next peice of code of is Training data than testing data data and finally exporting the model 

In [10]:
# IMAGES_PATH = "rps_data_sample"
# data = gesture_recognizer.Dataset.from_folder(
#     dirname=IMAGES_PATH,
#     hparams=gesture_recognizer.HandDataPreprocessingParams()
# )
# # Split the archive into training, validation and test dataset.
# train_data, rest_data = data.split(0.8)
# validation_data, test_data = rest_data.split(0.5)

In [11]:
# hparams = gesture_recognizer.HParams(export_dir="rock_paper_scissors_model")
# options = gesture_recognizer.GestureRecognizerOptions(hparams=hparams)
# model = gesture_recognizer.GestureRecognizer.create(
#     train_data=train_data,
#     validation_data=validation_data,
#     options=options
# )

In [12]:
# loss, acc = model.evaluate(test_data, batch_size=1)
# print(f"Test loss:{loss}, Test accuracy:{acc}")

In [13]:
# model.export_model()

The code below is for drawing landmarks on real time frame

In [14]:
mp_holistic = mp.solutions.holistic

mp_drawing = mp.solutions.drawing_utils
def hand_detector(frame,holistic):
    frame_rgb = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
    results = holistic.process(frame_rgb)

    ## for drawing left hand
    if results.left_hand_landmarks:
        mp_drawing.draw_landmarks(frame, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(255, 0, 0), thickness=2, circle_radius=4),
                                  mp_drawing.DrawingSpec(color=(255, 255, 255), thickness=2, circle_radius=2))
        
    ## From drawing right hand
    if results.right_hand_landmarks:
        mp_drawing.draw_landmarks(frame, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(255, 0, 0), thickness=2, circle_radius=4),
                                  mp_drawing.DrawingSpec(color=(255, 255, 255), thickness=2, circle_radius=2))
    
        
    return frame

The code bellow is load the model and use it for real-time and making it ready for real time camera application

In [15]:
BaseOptions = mp.tasks.BaseOptions
GestureRecognizer = mp.tasks.vision.GestureRecognizer
GestureRecognizerOptions = mp.tasks.vision.GestureRecognizerOptions
GestureRecognizerResult = mp.tasks.vision.GestureRecognizerResult
VisionRunningMode = mp.tasks.vision.RunningMode
result_name = [None,0]
def print_result(result: GestureRecognizerResult, output_image: mp.Image, timestamp_ms: int):
     #print('gesture recognition result: {}'.format(result))
     if len(result.gestures) and len(result.gestures[0]):
      gesture = result.gestures[0][0]
      if result_name[0] != gesture.category_name:
         result_name[0] = gesture.category_name
         result_name[1] = 0
      else:
         result_name[1] += 1
      print(gesture.category_name,result_name[1])
        
     else:
        result_name[0] = None
        result_name[1] = 0



    

options = GestureRecognizerOptions(
    base_options=BaseOptions(model_asset_path='exported_model/gesture_recognizer.task'),
    running_mode=VisionRunningMode.LIVE_STREAM,result_callback=print_result
    )
recognizer =  GestureRecognizer.create_from_options(options) 
timestamp = 0

I0000 00:00:1701148062.691524    3792 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1701148062.692408    4802 gl_context.cc:344] GL version: 3.2 (OpenGL ES 3.2 Mesa 23.0.4-0ubuntu1~22.04.1), renderer: Mesa Intel(R) Xe Graphics (TGL GT2)
W0000 00:00:1701148062.692685    3792 gesture_recognizer_graph.cc:129] Hand Gesture Recognizer contains CPU only ops. Sets HandGestureRecognizerGraph acceleration to Xnnpack.


In [16]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils
holistic = mp_holistic.Holistic(min_detection_confidence=0.7, min_tracking_confidence=0.8)
cap = cv.VideoCapture(0)

cap.set(cv.CAP_PROP_FRAME_HEIGHT,720)
cap.set(cv.CAP_PROP_FRAME_WIDTH,1080)
while cap.isOpened():
    timestamp += 1
    ret, frame = cap.read()
    if not ret:
        continue

    mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame)
    recognition_result = recognizer.recognize_async(mp_image,timestamp)
    #print(result_name[0])

    frame = hand_detector(frame,holistic)

    cv.imshow("Hand Sign Detection", frame )    
    if cv.waitKey(1) & 0xFF == ord('q') or (result_name[0] == "scissors" and result_name[1] > 6):
        break

    if result_name[0] == "paper" and result_name[1] > 6:
        screenshot = py.screenshot()
        screenshot.save("screenshot.png")
    
        
cap.release()
cv.destroyAllWindows()


I0000 00:00:1701148062.742867    3792 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1701148062.743628    4821 gl_context.cc:344] GL version: 3.2 (OpenGL ES 3.2 Mesa 23.0.4-0ubuntu1~22.04.1), renderer: Mesa Intel(R) Xe Graphics (TGL GT2)


U 0
L 0
L 1
L 2
L 3
L 4
L 5
L 6
L 7
L 8
L 9
L 10
L 11
L 12
L 13
L 14
L 15
L 16
L 17
L 18
L 19
L 20
L 21
L 22
L 23
L 24
L 25
L 26
L 27
L 28
L 29
L 30
L 31
L 32
B 0
B 1
B 2
B 3
B 4
B 5
B 6
B 7
B 8
B 9
B 10
B 11
B 12
B 13
B 14
B 15
B 16
B 17
B 18
B 19
B 20
B 21
B 22
C 0
C 1
C 2
O 0
O 1
 0
A 0
 0
 1
A 0
A 1
A 2
A 3
A 4
A 5
 0
S 0
A 0
A 1
 0
 1
A 0
A 1
 0
A 0
 0
 1
 2
 3
 4
 5
 6
 7
 8
 9
 10
 11
 12
 13
 14
 15
 16
O 0
O 1
O 0
O 0
Q 0
V 0
V 1
V 2
V 3
V 4
V 5
V 6
V 7
 0
 1
 2
 3
V 0
V 1
V 2
V 3
V 4
V 5
V 6
V 7
V 8
V 9
V 10
V 11
V 12
V 13
V 14
V 15
V 16
V 17
V 18
V 19
V 20
V 21
 0
 1
V 0
 0
 1
V 0
V 1
 0
V 0
V 1
 0
 1
 2
 3
 4
U 0
 0
U 0
U 1
U 2
U 3
U 4
U 5
U 6
U 7
 0
E 0
E 1
S 0
S 1
F 0
F 1
F 2
F 3
F 4
F 5
F 6
F 7
F 8
F 9
F 10
F 11
O 0
G 0
G 1
G 2
L 0
G 0
L 0
L 1
F 0
F 1
F 2
F 3
V 0
L 0
L 1
L 2
 0
F 0
U 0
R 0
 0
C 0
F 0
C 0
D 0
 0
 1
F 0
 0
R 0
C 0
 0
F 0
 0
 1
 2
F 0
F 1
 0
 1
D 0
C 0
F 0
R 0
R 1
L 0
D 0
V 0
V 1
V 2
V 3
V 4
V 5
V 6
V 7
V 8
V 9
V 10
V 11
V 12
V 13
V 14
V 15
V 16
V 17
V 18
V